In [62]:
import pandas as pd
import numpy as np
from gmf import GMF
from mlp import MLP
from neumf import NeuMF
from neumf_social import NeuMF_Social
import torch.optim as optim
import math

In [63]:
data_path = "./data/yelp/"

In [64]:
train_ratings = list(np.load(data_path + "train_ratings.npy"))
val_ratings = list(np.load(data_path + "val_ratings.npy"))
test_ratings = list(np.load(data_path + "test_ratings.npy"))

In [65]:
social_embeddings_dict = np.load(data_path + "deepwalk_embeddings.npy").item()

In [66]:
yelp_rating_path = data_path + 'yelp.train.rating'
yelp_ratings = pd.read_csv(yelp_rating_path, sep='\t', header=None, names=['uid', 'rid', 'rating'],  engine='python')
user_id = yelp_ratings[['uid']].drop_duplicates()
item_id = yelp_ratings[['rid']].drop_duplicates()
num_users = user_id['uid'].max() - user_id['uid'].min() + 1
num_items = item_id['rid'].max() - item_id['rid'].min() + 1

In [82]:
import torch
from torch.utils import data

class Dataset(data.Dataset):
  'Characterizes a dataset for PyTorch'
  def __init__(self, ratings, social_embeddings_dict):
        'Initialization'
        self.ratings  = ratings
        self.social_embeddings = social_embeddings_dict
  def __len__(self):
        'Denotes the total number of samples'
        return len(self.ratings)

  def __getitem__(self, index):
        'Generates one sample of data'
       # # Load data and get label
        user_item_pair = self.ratings[index][0:2].astype('float')
        user_social = self.social_embeddings[user_item_pair[0]].astype('float')
        #user_social = np.zeros(64).astype('float')
        user_item_pair_social = np.concatenate((user_item_pair, user_social), axis=None)
        X = user_item_pair_social
        y = self.ratings[index][2:3].astype('float')
        return X, y

In [68]:
gmf_config = {'alias': 'train_gmf',
              'num_epoch': 8,
              'batch_size': 1024,
              'adam_lr': 1e-3,
              'num_users': num_users,
              'num_items': num_items,
              'latent_dim': 8,
              'l2_regularization': 0.01,
              'use_cuda': True,
              'device_id': 0}

In [69]:
mlp_config = {'alias': 'mlp_factor8neg4_bz256_166432168_pretrain_reg_0.0000001',
              'num_epoch': 5,
              'batch_size': 1024, 
              'num_users': num_users,
              'num_items': num_items,
              'latent_dim': 8,
              'layers': [16,32,16,8]}

In [70]:
neumf_config = {'alias': 'pretrain_neumf_factor8neg4',
                'num_epoch': 6,
                'batch_size': 1024,
                'num_users': num_users,
                'num_items': num_items,
                'latent_dim_mf': 8,
                'latent_dim_mlp': 8,
                'layers': [16,32,16,8],  # layers[0] is the concat of latent user vector & latent item vector
                'pretrain': False,
                'pretrain_mf': None,
                'pretrain_mlp': None
                }

In [71]:
neumf_social_config = {'alias': 'pretrain_neumf_factor8neg4',
                'num_epoch': 8,
                'batch_size': 1024,
                'num_users': num_users,
                'num_items': num_items,
                'latent_dim_mf': 8,
                'latent_dim_mlp': 8,
                'user_social_dim_in':64,
                'user_social_dim_out':16,
                'layers': [16+16,32,16,8],  # 32 layers[0] is the concat of latent user vector & latent item vector
                'pretrain': False,
                'pretrain_mf': None,
                'pretrain_mlp': None
                }

In [72]:
training_params = {'batch_size': 1024,'shuffle': True,'num_workers': 0}
training_set = Dataset(train_ratings, social_embeddings_dict)
train_generator = data.DataLoader(training_set, **training_params)
val_params = {'batch_size': 1024,'shuffle': True,'num_workers': 0}
val_set = Dataset(val_ratings, social_embeddings_dict)
val_generator = data.DataLoader(val_set, **val_params)
test_params = {'batch_size': 1024,'shuffle': True,'num_workers': 0}
test_set = Dataset(test_ratings, social_embeddings_dict)
test_generator = data.DataLoader(test_set, **test_params)

In [73]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [84]:
def epoch_run(model, generator, opt, criterion,mode="train", social=False):
    running_loss = 0
    if(mode == "train"):
        model.train()
    else:
        model.eval()
    for local_batch, local_labels in generator:
        local_batch_social  = torch.tensor(local_batch[:, 2:]).type(torch.float).to(device)
        local_batch  = torch.tensor(local_batch[:, 0:2]).type(torch.long).to(device)
        local_labels = local_labels.type(torch.float).to(device)
        #print(local_batch.size(), local_batch_social.size())
        y_preds = model(local_batch[:,0], local_batch[:,1], local_batch_social)
        loss = criterion(y_preds, local_labels)
        running_loss += (loss.item()*local_labels.size()[0])
        if(mode == "train"):
            opt.zero_grad()
            loss.backward()
            opt.step()
    avg_loss = running_loss * 1.0 / (len(generator.dataset))
    return avg_loss

In [75]:
def predict(model, generator):
    model.eval()
    y_preds_all = torch.Tensor().to(device) 
    y_labels_all = torch.Tensor().to(device) 
    for local_batch, local_labels in generator:
        local_batch_social  = torch.tensor(local_batch[:, 2:]).type(torch.float).to(device)
        local_batch  = torch.tensor(local_batch).type(torch.long).to(device)
        local_labels = local_labels.type(torch.float).to(device)
        with torch.no_grad():
            y_preds = model(local_batch[:,0], local_batch[:,1], local_batch_social)
        y_preds_all = torch.cat((y_preds_all,y_preds))
        y_labels_all = torch.cat((y_labels_all,local_labels))
    return y_preds_all, y_labels_all

In [76]:
import math
def evaluate(model, generator):
    y_preds_all, y_labels_all = predict(model, generator)  
    y_preds = list(y_preds_all.view(1, y_preds_all.size()[0]).to("cpu").numpy()[0])
    y_actuals = list(y_labels_all.view(1, y_labels_all.size()[0]).to("cpu").numpy()[0])
    #print(type(y_preds), type(y_actuals))
    tmse = sum([(a-b) * (a-b) for a,b in zip(y_preds, y_actuals)])
    rmse = math.sqrt((1.0*tmse)/len(y_preds))
    return rmse

In [77]:
def train_gmf():
    model = GMF(gmf_config).to(device)
    opt = optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999),weight_decay=1e-4)
    criterion = torch.nn.MSELoss()
    for epoch in range(gmf_config['num_epoch']):
        print("running epoch ", epoch)
        train_mse = epoch_run(model, train_generator, opt, criterion, "train")
        val_mse = epoch_run(model, val_generator, opt, criterion,"val")
        print("train mse loss => ", train_mse, "val mse loss => ", val_mse)
    return model

In [78]:
def train_mlp():
    model = MLP(mlp_config).to(device)
    opt = optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999),weight_decay=1e-4)
    criterion = torch.nn.MSELoss()
    for epoch in range(mlp_config['num_epoch']):
        print("running epoch ", epoch)
        train_mse = epoch_run(model, train_generator, opt, criterion,"train")
        val_mse = epoch_run(model, val_generator, opt,criterion, "val")
        print("train mse loss => ", train_mse, "val mse loss => ", val_mse)
    return model

In [79]:
def train_neumf():
    model = NeuMF(neumf_config).to(device)
#     if config['pretrain']:  #TODO:: Manoj
#         model.load_pretrain_weights()
    opt = optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999),weight_decay=1e-4)
    criterion = torch.nn.MSELoss()
    for epoch in range(neumf_config['num_epoch']):
        print("running epoch ", epoch)
        train_mse = epoch_run(model, train_generator, opt,criterion, "train")
        val_mse = epoch_run(model, val_generator, opt,criterion, "val")
        print("train mse loss => ", train_mse, "val mse loss => ", val_mse)
    return model

In [85]:
def train_neumf_social():
    model = NeuMF_Social(neumf_social_config).to(device)
#     if config['pretrain']:  #TODO:: Manoj
#         model.load_pretrain_weights()
    opt = optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999),weight_decay=1e-4)
    criterion = torch.nn.MSELoss()
    for epoch in range(neumf_social_config['num_epoch']):
        print("running epoch ", epoch)
        train_mse = epoch_run(model, train_generator, opt,criterion, "train")
        val_mse = epoch_run(model, val_generator, opt,criterion, "val")
        print("train mse loss => ", train_mse, "val mse loss => ", val_mse)
    return model

In [22]:
gmf_model = train_gmf()

running epoch  0


/home/forcerequestspring19_gmail_com/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


train mse loss =>  9.595478486335345 val mse loss =>  6.078701062905395
running epoch  1
train mse loss =>  4.1054407974208225 val mse loss =>  2.6434405063625026
running epoch  2
train mse loss =>  2.0464591744939744 val mse loss =>  1.6799269031932915
running epoch  3
train mse loss =>  1.6296891914957456 val mse loss =>  1.5921714111437466
running epoch  4
train mse loss =>  1.607191766190543 val mse loss =>  1.5916036133728044
running epoch  5
train mse loss =>  1.607115447435845 val mse loss =>  1.5916110017050518
running epoch  6
train mse loss =>  1.607120271086772 val mse loss =>  1.5916304270049932
running epoch  7
train mse loss =>  1.6071295327888098 val mse loss =>  1.5916016936755122


In [23]:
mlp_model = train_mlp()

running epoch  0


/home/forcerequestspring19_gmail_com/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


train mse loss =>  2.088761404724617 val mse loss =>  1.5956651328082017
running epoch  1
train mse loss =>  1.5162342781998275 val mse loss =>  1.4650156432668133
running epoch  2
train mse loss =>  1.2962417460996116 val mse loss =>  1.3326339979284552
running epoch  3
train mse loss =>  1.1947617218902533 val mse loss =>  1.3236104126429726
running epoch  4
train mse loss =>  1.1414955583747792 val mse loss =>  1.3322711227758965


In [81]:
neumf_model = train_neumf()

running epoch  0


/home/forcerequestspring19_gmail_com/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  
/home/forcerequestspring19_gmail_com/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


train mse loss =>  2.0925040843983993 val mse loss =>  1.826724066453473
running epoch  1
train mse loss =>  1.3736055020416948 val mse loss =>  1.5246680023304715
running epoch  2
train mse loss =>  1.1947567357797604 val mse loss =>  1.4640836185916488
running epoch  3
train mse loss =>  1.1542554382059538 val mse loss =>  1.452479722134367
running epoch  4
train mse loss =>  1.1299274069102143 val mse loss =>  1.4514484848971039
running epoch  5
train mse loss =>  1.101578748978171 val mse loss =>  1.455236423131443


In [86]:
neumf_social_model = train_neumf_social()

running epoch  0


/home/forcerequestspring19_gmail_com/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  
/home/forcerequestspring19_gmail_com/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


train mse loss =>  1.9654662543268306 val mse loss =>  1.8026614366485878
running epoch  1
train mse loss =>  1.324923334100735 val mse loss =>  1.5044690174713673
running epoch  2
train mse loss =>  1.1931316683679443 val mse loss =>  1.4641228097475312
running epoch  3
train mse loss =>  1.1663814631260732 val mse loss =>  1.4548150612595026
running epoch  4
train mse loss =>  1.149474050849452 val mse loss =>  1.4514291902213132
running epoch  5
train mse loss =>  1.1308336555022884 val mse loss =>  1.4514488434918573
running epoch  6
train mse loss =>  1.1113914707063364 val mse loss =>  1.4602239946964948
running epoch  7
train mse loss =>  1.092948677745897 val mse loss =>  1.4607861819481034


In [30]:
torch.save(gmf_model.state_dict(), "./saved_models/gmf.dict")

In [31]:
torch.save(mlp_model.state_dict(), "./saved_models/mlp.dict")

In [32]:
torch.save(neumf_model.state_dict(), "./saved_models/neumf.dict")

In [54]:
torch.save(neumf_social_model.state_dict(), "./saved_models/neumf_social.dict")

In [55]:
model = NeuMF_Social(neumf_social_config).to(device)
model.load_state_dict(torch.load("./saved_models/neumf_social.dict"))
model.eval()

NeuMF_Social(
  (embedding_user_mlp): Embedding(122824, 8)
  (embedding_item_mlp): Embedding(28007, 8)
  (embedding_user_mf): Embedding(122824, 8)
  (embedding_item_mf): Embedding(28007, 8)
  (social_mlp): Linear(in_features=64, out_features=16, bias=True)
  (fc_layers): ModuleList(
    (0): Linear(in_features=32, out_features=32, bias=True)
    (1): Linear(in_features=32, out_features=16, bias=True)
    (2): Linear(in_features=16, out_features=8, bias=True)
  )
  (affine_output): Linear(in_features=16, out_features=1, bias=True)
)

In [56]:
evaluate(model, test_generator)

/home/forcerequestspring19_gmail_com/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  
/home/forcerequestspring19_gmail_com/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  import sys


1.1533523724046224